In [1]:
# Standard Library Imports
import random
import math
import os
import sys
import math
import copy
import random
sys.path.append('..')

# External Library Imports
import numpy as np
from tqdm import trange
import matplotlib
import matplotlib.pyplot as plt

# Deep Learning and Bayesian Deep Learning Libraries
import deepbayes_prealpha
import deepbayes_prealpha.optimizers as optimizers
from deepbayes_prealpha import PosteriorModel
from deepbayes_prealpha.analyzers import prob_veri

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [2]:
pos_ind = 0
HORIZON = 10
momentum = -0.35
momentum_eps = 0.08
OBSTACLE = [0.4,0.4]
GOAL_STATE = [-0.2,-0.2]

In [3]:
"""
The handcrafted 2D point environment we consider
"""
class P2D_env:
    def __init__(self):
        self.x = 0.90  + random.uniform(-0.1, 0)
        self.y = 0.90  + random.uniform(-0.1, 0)  #np.random.normal(-0.05, 0.025)
        self.dx = momentum
        self.dy = momentum
        # Point Mass Dynamics Control:
        self.h = 0.35      # time resolution
        self.T = HORIZON       # Time for each trajectory
        self.eta = 0.75    # friction coef
        self.m = 1.0      # mass of the car
        self.t = 0        # current time

    def step(self, act):
        if(self.t > self.T):
            sys.exit(-1)
        pt = np.asarray([self.x, self.y])
        vt = np.asarray([self.dx, self.dy])
        ut = np.asarray(act)
        vt_1 = ((1-((self.h*self.eta)/self.m))*vt) + ((self.h/self.m)*ut)
        pt_1 = pt + (self.h*vt_1)
        if(np.linalg.norm(pt_1 - OBSTACLE) < 0.05):
            print("COLLIDED")
            self.dx, self.dy = 0, 0
        self.x, self.y = pt_1[0], pt_1[1]
        self.dx, self.dy = vt_1[0], vt_1[1]
        self.t += 1
        state = np.asarray([self.x, self.y, self.dx, self.dy])
        return state, np.linalg.norm(pt) - np.linalg.norm(pt_1), self.complete(), 0

    def complete(self):
        state = np.asarray([self.x, self.y])
        if(self.x < 0.05 and self.y < 0.05):
            return True
        if(self.x < 0.0 or self.y < 0.0):
            return True
        #if(np.linalg.norm(state) < 0.05):
        #    return True
        if(self.t==self.T):
            return True
    def reset(self):
        self.x = 0.90  + random.uniform(-0.3, 0)
        self.y = 0.90  + random.uniform(-0.3, 0)
        self.dx = momentum
        self.dy = momentum
        self.t = 0
        return np.asarray([self.x, self.y, self.dx, self.dy])
    def action_space_sample(self):
        return np.random.rand(2)
    def observation_space_sample(self):
        return np.random.rand(4)

In [4]:
env = P2D_env()
state_0 = env.reset()
initial_state = state_0
action_dims = len(env.action_space_sample())
observe_dims = len(env.observation_space_sample())
model_input_dims = observe_dims + action_dims
model_output_dims = observe_dims
control_input_dims = observe_dims
control_output_dims = action_dims

In [5]:
bayes_model = PosteriorModel("MinimalModel")
#bayes_model = PosteriorModel("LearnedSystem/BayesDyn")
bayes_model.posterior_var += 0.0000001

ACTION_MATRIX = np.load("MinimalAM.npy")
C_STATES = len(ACTION_MATRIX)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                84        
                                                                 
 dense_1 (Dense)             (None, 12)                156       
                                                                 
 dense_2 (Dense)             (None, 12)                156       
                                                                 
 dense_3 (Dense)             (None, 4)                 52        
                                                                 
Total params: 448
Trainable params: 448
Non-trainable params: 0
_________________________________________________________________
deepbayes: detected the above model 
 None


2023-03-02 21:20:56.502761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def return_AM_action(state_0, ACTION_MATRIX):
    #ind_i = math.floor(state_0[0]*C_STATES)
    #ind_j = math.floor(state_0[1]*C_STATES)
    ind_i = round(state_0[0]*C_STATES)
    ind_j = round(state_0[1]*C_STATES)
    a, b = random.uniform(-0.4, 0), random.uniform(-0.4, 0)
    retval = [a, b]
    try:
        retval =  ACTION_MATRIX[ind_i][ind_j]
    except:
        if(ind_i >= C_STATES*0.9 and ind_j >= C_STATES*0.9):
            retval = [-0, -0]
        elif(ind_j >= C_STATES*0.9):
            retval = ACTION_MATRIX[ind_i][C_STATES-1]
            retval[1] = -0
        elif(ind_j >= C_STATES*0.9):
            retval = ACTION_MATRIX[C_STATES-1][ind_j]
            retval[0] = -0
    return retval


def reward_function(state_0, dyn_pred, goal):
    state_0 = tf.cast(tf.squeeze(state_0), dtype=tf.float32)
    dyn_pred = tf.cast(tf.squeeze(dyn_pred), dtype=tf.float32)
    state_1 = state_0 + dyn_pred
    #print("State shape: ", state_1.shape)
    d1 = tf.norm(state_0[pos_ind:pos_ind+2] - goal, axis=0)
    d2 = tf.norm(state_1[pos_ind:pos_ind+2] - goal, axis=0)

    o1 = tf.norm(state_0[pos_ind:pos_ind+2] - OBSTACLE, axis=0)
    o2 = tf.norm(state_1[pos_ind:pos_ind+2] - OBSTACLE, axis=0)
    return (-1*(d2-d1)) + ((0.1/o2)*(o2-o1))

    #return (-1*(d2-d1))

# Lets try to do a full IBP pass through these networks
def verify_state(state, eps, predicate, p_arr, refine=2.5, s=False):

    s0 = np.asarray(state)                              # Lower bound of the states
    s1 = s0 + eps                                       # Add epsilon to the postion to get the upper bound
    input_to_alg = (s0, s1)

    initial_state[pos_ind:pos_ind+2] = s0               # Making copies of the state with velocity to pass to bounds
    s0_pre = copy.deepcopy(initial_state)
    initial_state[pos_ind:pos_ind+2] = s1
    s1_pre = copy.deepcopy(initial_state)
    s0_pre[pos_ind+2:pos_ind+4] -= 2*eps                # Subtracting epsilon from momentum to get interval over velocity
    diff = s0_pre - s1_pre

    # Propagating the intervals through the controller to get action intervals
    act = return_AM_action(s0_pre, ACTION_MATRIX) #control_model.predict(np.asarray([s0_pre]))
    act_l = np.squeeze(act)
    act_u = np.squeeze(act)

    # Adding the action intervals to state intervals to get full intervals
    s0 = np.concatenate((s0_pre,act_l))
    s1 = np.concatenate((s1_pre,act_u))

    s0 = np.asarray([s0])
    s1 = np.asarray([s1])
    #p, outs = prob_veri(bayes_model, s0, s1, w_marg=2.5, samples=3, predicate=predicate, depth=3, y_scaler=None)
    p, outs = prob_veri(bayes_model, s0, s1, w_marg=2.5, samples=2, predicate=predicate, depth=2, y_scaler=None)
    if(p == 0):
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print("~~~~~NOT REACHABLE~~~~~~~~")
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~")
        return 0.00000000000000000001

    #Get max and mins here!
    mins = [i[0] for i in outs]
    maxs = [i[1] for i in outs]
    try:
        mins = np.min(mins, axis=0)[pos_ind:pos_ind+2]
        maxs = np.max(maxs, axis=0)[pos_ind:pos_ind+2]
        maxs[0] = min(maxs[0], np.squeeze(s1)[0])
        maxs[1] = min(maxs[1], np.squeeze(s1)[1])
    except:
        mins = np.asarray(mins)
        maxs = np.asarray(maxs)
    # Account for first iteration here:
    if((maxs < INIT_SAFE_ZONE).all()):
        print("One step reachable. (p=%s)"%(p))
        return p
    else:
        mi_i, ma_i = math.floor(mins[0]/eps), math.ceil(maxs[0]/eps)
        #mi_j, ma_j = math.floor((mins[1]+0.5)/eps), math.ceil((maxs[1]+0.5)/eps)
        mi_j, ma_j = math.floor((mins[1])/eps), math.ceil((maxs[1])/eps)
        # index clipping
        mi_i, ma_i = max(0, mi_i), max(0, ma_i)
        mi_j, ma_j = max(0, mi_j), max(0, ma_j)
        mi_i, ma_i = min(len(p_arr), mi_i), min(len(p_arr), ma_i)-1
        mi_j, ma_j = min(len(p_arr), mi_j), min(len(p_arr), ma_j)-1
        mi_i, ma_i = max(0, mi_i), max(0, ma_i)
        mi_j, ma_j = max(0, mi_j), max(0, ma_j)
        #mi_j, ma_j = -1*mi_j, -1*ma_j
        print("X ", mi_i, ma_i)
        print("Y ", mi_j, ma_j)
        if(mi_i == 0 and ma_i == 0):
            ma_i += 1
        if(mi_j == 0 and ma_j == 0):
            ma_j += 1
        try:
            worst_future = np.min(np.asarray(p_arr[mi_i:ma_i,mi_j:ma_j]).flatten())
        except:
            worst_future = 0.0 #p_arr[mi_i,mi_j]
        print("Multi-step reachable. Worst case: %s (p=%s)"%(worst_future, p))
        print(mins, maxs)
        if(worst_future == 0.0):
            return 0.00000000000000000001
        if(p == 0):
            return 0.00000000000000000001
        return p * worst_future


SAFE_REGION = 0.2
INIT_SAFE_ZONE = 0.2

def label(state_l, state_u):
    collision = True
    # Overlap, side
    if(not (state_u[0] <= (OBSTACLE[0]-0.025) or state_l[0] >= (OBSTACLE[0]+0.025)
            or state_u[1] <= (OBSTACLE[1] - 0.025) or state_l[1] >= (OBSTACLE[1] + 0.025))):
        collision = False
    # Obstacle subsumed
    if(state_u[0] < (OBSTACLE[0]+0.025) and state_l[0] > (OBSTACLE[0]-0.025) and 
          state_u[1] < (OBSTACLE[1]+0.025) and state_l[1] > (OBSTACLE[1]-0.025)):
        collision = False
    # State subsumed
    if(state_u[0] > (OBSTACLE[0]+0.025) and state_l[0] < (OBSTACLE[0]-0.025) and 
          state_u[1] > (OBSTACLE[1]+0.025) and state_l[1] < (OBSTACLE[1]-0.025)):
        collision = False
    # Point intersection
    def point_intersection(low, high, pt) :
        if (pt[0] > low[0] and pt[0] < high[0] and pt[1] > low[1] and pt[1] < high[1]):
            return False
        else:
            return True
    pts = [state_l, state_u, [state_l[0], state_u[1]], [state_u[0], state_l[1]]]
    for pt in pts:
        collision = (collision and point_intersection(OBSTACLE, np.asarray(OBSTACLE)+0.025, pt))
    return collision


def predicate(source_l, source_u, state_l, state_u):
    source_l = source_l[0:observe_dims]
    source_u = source_u[0:observe_dims]

    # Start by assuming we are safe wrt the obstacle collision critera
    collision = True

    # Check if we overlap with the obstacle, if so we are unssafe
    collision = collision and label(source_l, source_u)
    collision = collision and label(state_l, state_u)

    # Check that we make it to the safe region in the worst case
    goal = (state_u[pos_ind:pos_ind+2] <= SAFE_REGION).all()

    # Check that the velocity constraint is satisfied
    velo = (abs(state_u[pos_ind+2:pos_ind+4] - momentum) <= 2*momentum_eps).all()
    #print(abs(state_u[pos_ind+2:pos_ind+4] - momentum), velo)
    print(state_u[pos_ind:pos_ind+2] )
    return collision and goal and velo



In [7]:
from tqdm import tqdm
INIT_SAFE_REGION = 0.2
states = 30
end_point = 0.50
eps = end_point/(states-1)
non_zero = []
ind_i, ind_j = 0,0
multiplier = 1.0
global_probas = np.zeros((states,states))
for k in trange(states-int(SAFE_REGION/eps) + 1):
    probas = []
    for i in np.linspace(0, end_point, num=states):
        for j in tqdm(np.linspace(0, end_point, num=states)):
            if(global_probas[ind_i][ind_j] != 0):
                probas.append(global_probas[ind_i][ind_j])
                ind_j += 1; continue
            elif((i < INIT_SAFE_REGION and j < INIT_SAFE_REGION)):
                probas.append(1.0)
                ind_j += 1; continue
            if(i > (SAFE_REGION + eps/1.5) or j > (SAFE_REGION + eps/1.5)):
                probas.append(0.0)
                ind_j += 1; continue
            print("State: (%s, %s) (eps: %s)"%(i,j,eps))
            p = verify_state([i, j], eps, predicate, global_probas, refine=2.0, s=False)

            print(" ")
            print(" ")
            print("Probability: %s"%(p))
            print(" ")
            print(" ")

            probas.append(p)
            non_zero.append(p)
            ind_j += 1
        ind_i += 1
        ind_j = 0
    SAFE_REGION += eps
    ind_i = 0
    probas = np.reshape(probas, (states,states))
    probas_vals = np.argwhere(probas > 0.00)
    global_probas[probas_vals] = probas[probas_vals]
    print("======================================")
    print("==========EXPANDING GOAL==============")
    print("======================================")
    probas = np.asarray(probas)
    probas = probas.reshape(states,states)
    #np.save("probs_P2D_v0_%s_%s"%(states, eps), probas)

probas = np.asarray(probas)
probas = probas.reshape(states,states)
#np.save("probs_P2D_v0_%s_%s"%(states, eps), probas)

  0%|                                                                                                                                  | 0/30 [00:00<?, ?it/s]

State: (0.0, 0.20689655172413793) (eps: 0.017241379310344827)




Checking Samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 106.19it/s]


[-0.04070623  0.14980705]
[-0.04381882  0.14576824]
Found 2 safe intervals
About to compute intersection for this many intervals:  2




Computing intersection weights:   0%|                                                                                                   | 0/2 [00:00<?, ?it/s]/Users/matthewwicker/Desktop/Development/BNNReachAvoid/minimalPuck2Dv1/../deepbayes_prealpha/analyzers/probverification.py:425: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  stage1_args.append((model.posterior_mean, model.posterior_var, np.swapaxes(np.asarray([weight_intervals[wi]]),1,0), margin, verbose, n_proc, False))
Computing intersection weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2405.68it/s]

...




  0%|                                                                                                                                   | 0/2 [00:00<?, ?it/s]2023-03-02 21:21:16.414767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 21:21:16.414767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00

Depth 1 prob:  1.7739561128702794
Depth 2 has 1 intersections




  0%|                                                                                                                                   | 0/1 [00:00<?, ?it/s]Process SpawnPoolWorker-239:
Process SpawnPoolWorker-235:
Process SpawnPoolWorker-240:
Traceback (most recent call last):
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return se

  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process SpawnPoolWorker-222:
Traceback (most recent call last):
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/matthewwicker/opt/anaconda

Process SpawnPoolWorker-243:
Process SpawnPoolWorker-242:
Process SpawnPoolWorker-241:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/__init__.py", line 16, in <module>
    from . import context
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/context.py", line 6, in <module>
Traceback (most recent call last):
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/matthewwicker/opt/anaconda3/envs/bayesdev/lib/python3.10/multiprocessing/queues.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

fig= plt.figure(num=None, figsize=(6, 6), dpi=100)
arr = probas
arr = np.rot90(np.fliplr(arr))
arr = np.rot90(arr)
plt.imshow(arr, cmap='YlGnBu')
plt.colorbar()
plt.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

c_size = 25

ACTION_MATRIX = np.load("MinimalAM.npy")
ACTIONMATRIX = np.rot90(np.fliplr(ACTION_MATRIX))
locations = np.zeros((c_size,c_size))
ydirections = np.zeros((c_size,c_size))
xdirections = np.zeros((c_size,c_size))
sizes = np.ones((c_size,c_size))
ind_i, ind_j = 0, 0 
for i in np.linspace(0,1,c_size):
    ind_j = 0
    for j in np.linspace(0,1,c_size):
        arr = ACTIONMATRIX[ind_i][ind_j]
        xdirections[ind_i][ind_j] = arr[0]
        ydirections[ind_i][ind_j] = arr[1]
        ind_j += 1
    ind_i += 1
    

def vector_to_rgb(angle, absolute):
    """Get the rgb value for the given `angle` and the `absolute` value

    Parameters
    ----------
    angle : float
        The angle in radians
    absolute : float
        The absolute value of the gradient
    
    Returns
    -------
    array_like
        The rgb value as a tuple with values [0..1]
    """
    global max_abs

    # normalize angle
    angle = angle % (2 * np.pi)
    if angle < 0:
        angle += 2 * np.pi
    c = (angle / 2 / np.pi, absolute / max_abs, absolute / max_abs)
    cmap = matplotlib.cm.get_cmap('winter')
    #cmap = sns.color_palette("blend:#C38D9E,#E27D60,#E8A87C,#85DCBA,#41B3A4", as_cmap=True)
    c = np.clip(np.asarray(cmap((angle / 2 / np.pi))) + 0.1, 0,1)
    return c
    #return matplotlib.colors.hsv_to_rgb(c)

angles = np.arctan2(xdirections, ydirections)
lengths = np.sqrt(np.square(ydirections) + np.square(xdirections))
max_abs = np.max(lengths)
c = np.array(list(map(vector_to_rgb, angles.flatten(), 
                                      np.ones_like(lengths.flatten()) * max_abs)))


fig= plt.figure(num=None, figsize=(6, 6), dpi=120)
ax = plt.gca()
# ============================================================
# ============================================================
# ============================================================
plt.quiver(xdirections, ydirections, color=c, scale=15, headlength=7, linewidth=1, edgecolor=c)  
# ============================================================
# ============================================================
# ============================================================
rect1 = patches.Rectangle((0,0),6,6,linewidth=8,edgecolor='#47DC3D',facecolor='none', alpha=0.3)
ax.add_patch(rect1)

rect1 = patches.Rectangle((11,11),2,2,linewidth=8,edgecolor='#ED2939',facecolor='none', alpha=0.3)
ax.add_patch(rect1)

a_color = "#25274D"
ax.spines['bottom'].set_color(a_color)
ax.spines['top'].set_color(a_color)
ax.spines['left'].set_color(a_color)
ax.spines['right'].set_color(a_color)
ax.xaxis.label.set_color(a_color)
ax.tick_params(axis='x', colors=a_color)
ax.tick_params(axis='y', colors=a_color)
#sns.set(rc={'axes.facecolor':f_color})
ax.grid(False)
plt.title('Learned Actions (Three Layer)', y=1.0125, color=a_color, fontsize=20)
ax.set_xticklabels([0.0, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels([0.0, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
plt.show()

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

probas = np.load('threelayerprobas-extra.npy')

fig= plt.figure(num=None, figsize=(6, 6), dpi=150)
ax = plt.gca()
arr = probas
arr = np.rot90(np.fliplr(arr))
arr = np.rot90(arr)
plt.imshow(arr, cmap='YlGnBu')
plt.colorbar(fraction=0.046, pad=0.04)
a_color = "#25274D"
ax.spines['bottom'].set_color(a_color)
ax.spines['top'].set_color(a_color)
ax.spines['left'].set_color(a_color)
ax.spines['right'].set_color(a_color)
ax.xaxis.label.set_color(a_color)
ax.tick_params(axis='x', colors=a_color)
ax.tick_params(axis='y', colors=a_color)
#sns.set(rc={'axes.facecolor':f_color})
ax.grid(False)
plt.title('Reach-Avoid Prob. (3 Layer, Refined Params)', y=1.025, color=a_color, fontsize=18)
ax.set_xticklabels((np.asarray([0, 0, 10, 20, 30, 40, 50])/100).round(2), fontsize=20)
ax.set_yticklabels(reversed((np.asarray([0, 0, 10, 20, 30, 40, 50, 50])/100).round(2)), fontsize=20)
plt.xlabel('', fontsize=26)
plt.ylabel('', fontsize=26)
plt.show()